In [1]:
from project_lib import *

In [2]:
df = pd.read_csv('dataset_tmc.csv', parse_dates = ['Dates'], date_format = '%d/%m/%y')
df.columns = [i.lower() for i in df.columns]
df.columns = [i.replace(' ','_') for i in df.columns]
df = df.set_index('dates')

# Check NaNs
if df.isna().sum().sum() != 0:
    print('check NaNs in data')

# Drop Sat&Sun if any
df = df[df.index.dayofweek<5]

# as convention 1 week is 5 trading days and 1 month is 21 trading days so that each year has 252 trading days

In [33]:
# STRATEGY PARAMETERS
TRADED_SECURITIES = ['oil'
            # 'us_staples', 'eu_healthcare',
            # 'japan', 'brazil', 'nasdaq', 'us_2y',
            # 'us_15y', 'germany_10y', 'uk_15y', 'us_hy', 'oil', 'ind_metals', 'agri',
            # 'gold', 'silver', 'eurusd', 'usdjpy', 'chfjpy', 'eurbrl', 'gbpmxn'
    ]

LONG_PARAMS = {10:0.05, 21:0.15, 63:0.25} # dict with lenght of window as key and percentile for longs as values
SHORT_PARAMS = {10:0.95, 21:0.85, 63:0.75} # dict with lenght of window as key and percentile for shorts as values
MA_FAST_WDW = 20
MA_SLOW_WDW = 60

SUPPORTIVE_PCTL_MOVE = 0.3
COUNTER_PCTL_MOVE = 0.2
TRADES_MAX_DAYS = 21

MAX_DOLLAR_LOSS = 100


In [193]:
# list to store all trades
trades_list = []
trades_pnl = {}

for security_id in TRADED_SECURITIES:
    # generate signal based on standard strategy
    signal_all = meanrev_signal(df[security_id],
                                long_params = LONG_PARAMS,
                                short_params = SHORT_PARAMS,
                                ma_fast_wdw = MA_FAST_WDW,
                                ma_slow_wdw = MA_SLOW_WDW
                )

    # "start your backtest at t-10". hard coding initial date
    signal_all = signal_all.loc['2014-02-12':'2018-01-01']    ##############################################################DROPPARE LA DATA DI FINE

    # extract from all signal only the actual open buy/sell triggers
    signal_do = signal_all[signal_all!= 0]

    # iterates over all buy/sell signal and execute orders
    for dt_open, direction in zip(signal_do.index, signal_do):
        
        # create a unique id for each trade
        trade_id = security_id+'#'+str(dt_open)[:10]

        # price at which the trade is open
        price_open = df.loc[dt_open, security_id]

        # compute TP/SL returns and prices
        tp_return, sl_return = tp_sl_rule(df[security_id],
                                            dt_open,
                                            direction,
                                            supportive_pctl_move = SUPPORTIVE_PCTL_MOVE,
                                            counter_pctl_move = COUNTER_PCTL_MOVE
                                          )
        if direction == 1:
            price_tp = price_open * (1+tp_return)
            price_sl = price_open * (1+sl_return)
        elif direction == -1:
            price_tp = price_open * (1 - tp_return)
            price_sl = price_open * (1 - sl_return)

        # compute the optimal sizing such that all trades loses the same amount of $ if SL is hitted
        quantity = MAX_DOLLAR_LOSS/((price_open - price_sl)*direction)

        # store all trade info in df
        trade = pd.DataFrame({'security_id':security_id,
              'dt_open':str(dt_open)[:10],
              'price_open':price_open,
              'direction':direction,
              'quantity':quantity,
              'price_tp':price_tp,
              'price_sl':price_sl}, index=[trade_id])

        # create a temporary df that contains prices of instrument during trade
        dt_open_idxdf = list(df.index).index(dt_open)
        temp_px = df[[security_id]].iloc[dt_open_idxdf + 1: dt_open_idxdf + TRADES_MAX_DAYS + 1]
        temp_px['price_open'] = price_open
        temp_px['direction'] = direction
        temp_px['quantity'] = quantity
        temp_px['price_tp'] = price_tp
        temp_px['price_sl'] = price_sl

        # Check if and when a TP/SL is triggered and cut the temporary df accordingly
        if direction==1:
            temp_px['tp_hitted'] = (temp_px[security_id] > temp_px['price_tp']) * 1
            temp_px['sl_hitted'] = (temp_px[security_id] < temp_px['price_sl']) * 1
        elif direction==-1:
            temp_px['tp_hitted'] = (temp_px[security_id] < temp_px['price_tp']) * 1
            temp_px['sl_hitted'] = (temp_px[security_id] > temp_px['price_sl']) * 1

        temp_px['tp_sl_hitted'] = temp_px['tp_hitted'] + temp_px['sl_hitted']

        if 1 in list(temp_px['tp_sl_hitted']):
            dt_close = str(temp_px[temp_px['tp_sl_hitted']==1].index[:1][0])[:10]
        else:
            dt_close = str(list(temp_px.index)[-1:][0])[:10]

        temp_px = temp_px.loc[:dt_close]
        price_close = temp_px.loc[dt_close, security_id]

        # add closing trade date and price to trade df
        trade['dt_close'] = dt_close
        trade['price_close'] = price_close
        trade['duration'] = len(temp_px)

        












        trades_list.append(trade)


trades_list = pd.concat(trades_list)

In [194]:
security_id, str(dt_open)[:10], price_open, direction, quantity, price_tp, price_sl

type(price_sl)

numpy.float64

In [196]:
trades_list['return'] = trade_return(trades_list)

In [198]:
trades_list.describe()

,price_open,direction,quantity,price_tp,price_sl,price_close,duration,return
count,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000
mean,144.039277,0.256198,42.279083,144.531499,143.325434,142.581826,4.801653,-0.008619
std,78.899764,0.968628,13.747953,78.728350,79.245233,78.417218,4.498261,0.037843
min,63.680000,-1.000000,19.725024,65.319659,62.141917,63.680000,1.000000,-0.087406
25%,85.140000,-1.000000,30.165619,84.412870,84.879448,84.760000,2.000000,-0.032036
50%,105.160000,1.000000,43.448927,107.875054,102.832061,105.320000,3.000000,-0.024431
75%,173.700000,1.000000,53.862551,177.777696,171.484512,175.620000,7.000000,0.029202
max,314.560000,1.000000,66.134457,307.725363,319.629702,312.640000,21.000000,0.116894
